## Chiffrement et déchiffrement avec l'api de Pandas

**importation des packages**

In [1]:
import os
from Crypto.PublicKey import RSA
from cryptopandas import pandas as pd
import numpy as np

**Paths vers les clés**

In [2]:
path_store_key = ''
path_private_key = os.path.join(path_store_key, "private.pem")
path_public_key = os.path.join(path_store_key, "receiver.pem")
secret_code = "Unguessable" # Code secret pour lire la clé privée

**Chiffrer le code secret avec l'algorithme RSA**
* Les données sont chiffrées avec l'algorithme AES et un mot de passe. Ce mot de passe, chiffré avec l'algorithme RSA est stocké dans le fichier des données chiffrées.
* Une clé publique est utilisée pour chiffrer le mot de passe.
* Le destinataire recevra la clé privée pour déchiffrer le mot de passe. Celle-ci est accompagnée d'un code secret pour la lecture.
* Lors du déchiffrement :
   * Lire le mot de passe puis le déchiffrer avec la clé privée
   * Utiliser le mot de passe pour déchiffrer les données

In [3]:
# Generer la clé
key = RSA.generate(2048)

# Sauvegarder la clé privée. Il faudra disposer du code secret pour la lecture
private_key = key.exportKey(passphrase=secret_code)
outfile = open(path_private_key, "wb")
outfile.write(private_key)
outfile.close()

# Sauvegader la clé pour chiffrer le code secret
public_key = key.publickey().exportKey()
outfile = open(path_public_key, "wb")
outfile.write(public_key)
outfile.close()

**Créer un dataFrame**

In [4]:
data = {
    'ID' : ['001', '002', '003', '004'],
    'DEPARTEMENT' : ["Marketing", "Marketing", "Informatique", "Ventes"],
    'NOM' : ['Arthur Rimbeaud', 'Aurore Dupond', 'Victor Hugo', 'Marcel Pagnol'],
    'AGE' : [27, 35, 30, 25],
    'SALAIRE': [3000, 5000, 4000, 2500]
}

dtype = {
    'ID' : str,
    'DEPARTEMENT' : str,
    'NOM' : str,
    'AGE' : np.int64,
    'SALAIRE': np.int64
}

data = pd.DataFrame(data)
data

,ID,DEPARTEMENT,NOM,AGE,SALAIRE
0,001,Marketing,Arthur Rimbeaud,27,3000
1,002,Marketing,Aurore Dupond,35,5000
2,003,Informatique,Victor Hugo,30,4000
3,004,Ventes,Marcel Pagnol,25,2500


In [5]:
data.dtypes

ID             object
DEPARTEMENT    object
NOM            object
AGE             int64
SALAIRE         int64
dtype: object

**Chiffrer et stocker le dataframe**

In [6]:
recipient_key = RSA.importKey(open(path_public_key).read())
password = b'ungessaaaaaaable'
data.crypto.to_encrypt_file(path_file='data.ind',
                            recipient_key=recipient_key,
                            password=password)

**Lire et déchiffrer un fichier pour le transformer en dataframe**

In [7]:
private_key = RSA.importKey(open(path_private_key).read(), passphrase=secret_code)
data_ = pd.read_decrypt(path_file='data.ind',
                        dtype=dtype,
                        private_key=private_key)

In [8]:
data.equals(data_)

True